<a href="https://colab.research.google.com/github/tgeral68/TP-1-chatbot/blob/main/2-AnswerGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from matplotlib import pyplot as plt
from collections import Counter

In [6]:

from datasets import load_dataset
dataset = load_dataset("multi_woz_v22", trust_remote_code=True)

Generating train split:   0%|          | 0/8437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

mybpe = AutoTokenizer.from_pretrained("openai-community/gpt2")

In [8]:
class WoZWindowedGenerationDataset:
    def __init__(self, dataset, window_size=3):
        self.dataset = dataset
        self.window_size = window_size
        self.index = []
        for i, dial in enumerate(dataset):
            for j, speaker in enumerate(dial['turns']['speaker']):
                if speaker == 1:
                    self.index.append((i,j))
    def __len__(self):
        return len(self.index)

    def __getitem__(self, index):
        i, j = self.index[index]
        dial = self.dataset[i]['turns']['utterance']
        history = dial[max(j-self.window_size, 0):j]
        answer = dial[j]
        return {'history': history,
                'answer': answer}
class DialogueCollator(Dataset):
    def __init__(self, tokenizer, pad_value=mybpe.vocab):
        self.tokenizer = tokenizer
        self.tokenizer.add_special_tokens({'pad_token': '[PAD]', 'sep_token': '[sep]'})
        self.pad_value = self.tokenizer.pad_token_id

    def __call__(self, data):
        history = self.tokenizer([" [sep]".join(d['history']) for d in data],
                                 return_tensors='pt', return_length=True, padding=True)
        answer =  self.tokenizer(["start " + d['answer'] for d in data],
                                 return_tensors='pt', return_length=True, padding=True)

        return {
            'history': history.input_ids,
            'history_len': history.length,
            'answer': answer.input_ids,
            'answer_len': answer.length
        }


NameError: name 'Dataset' is not defined

In [ ]:
training_set = WoZWindowedGenerationDataset(dataset['train'])
collator = DialogueCollator(mybpe)
embeddings = nn.Embedding(len(mybpe.vocab) + 1 , 300, padding_idx=len(mybpe.vocab))

In [ ]:

class Seq2SeqRNN(nn.Module):
    def __init__(self,
            input_size=50,
            hidden_size=200,
            output_vocab_size=17,
            num_layers=1,
            bidirectional=False
        ):
        super().__init__()
        self.D = 2 if bidirectional else 1

        self.encoder = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                           	   batch_first=True, bidirectional=bidirectional)
        self.decoder = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                           	   batch_first=True, bidirectional=False)
        self.vocab_classifier = nn.Linear(self.D * hidden_size, output_vocab_size)
        self.h_init = nn.Parameter(torch.randn(num_layers  * self.D, hidden_size))
        self.c_init = nn.Parameter(torch.randn(num_layers * self.D, hidden_size))

    ''' RNN forward
        input : dict
            batched dat
    '''
    def forward(self, input_embeds, output_embeds, input_seq_len, output_seq_len, mask=None):
        ### ensure that no grad on the padded part of the sequences
        N, S = input_embeds.shape[0], input_embeds.shape[1]
        input_seq_len = input_seq_len.cpu()
        output_seq_len = output_seq_len.cpu()
        x = torch.nn.utils.rnn.pack_padded_sequence(input_embeds, input_seq_len, batch_first=True, enforce_sorted=False)

        ### forward through the rnn
        x_out, (h_out, c_out) = self.encoder(x,
                                    (
                                    self.h_init.unsqueeze(1).repeat(1, N, 1),
                                    self.c_init.unsqueeze(1).repeat(1, N, 1)
                                    )
                               )


        x_out, _ = torch.nn.utils.rnn.pad_packed_sequence(x_out, batch_first=True)
        ### get last ouput

        y = torch.nn.utils.rnn.pack_padded_sequence(output_embeds, output_seq_len, batch_first=True, enforce_sorted=False)

        y_out, (h_out, c_out) = self.decoder(y,
                                    (
                                    h_out,
                                    c_out
                                    )
                               )
        y_out, _ = torch.nn.utils.rnn.pad_packed_sequence(y_out, batch_first=True)


        return {"output": y_out}

    def encode(self, input_embeds, input_seq_len):
        ### ensure that no grad on the padded part of the sequences
        N, S = input_embeds.shape[0], input_embeds.shape[1]
        input_seq_len = input_seq_len.cpu()
        x = torch.nn.utils.rnn.pack_padded_sequence(input_embeds, input_seq_len, batch_first=True, enforce_sorted=False)

        ### forward through the rnn
        x_out, (h_out, c_out) = self.encoder(x,
                                    (
                                    self.h_init.unsqueeze(1).repeat(1, N, 1),
                                    self.c_init.unsqueeze(1).repeat(1, N, 1)
                                    )
                               )
        x_out, _ = torch.nn.utils.rnn.pad_packed_sequence(x_out, batch_first=True)
        return h_out, c_out

    def decode(self, input_embed, h_in, c_in):
        y = torch.nn.utils.rnn.pack_padded_sequence(input_embed, torch.ones(len(input_embed)), batch_first=True, enforce_sorted=False)

        y_out, (h_out, c_out) = self.decoder(y,
                                    (
                                    h_in,
                                    c_in
                                    )
                               )
        y_out, _ = torch.nn.utils.rnn.pad_packed_sequence(y_out, batch_first=True)

        return h_out, c_out



In [ ]:
class MyModel(nn.Module):
    def __init__(self, embedding):
        super().__init__()
        self.embedding = embedding
        self.model = Seq2SeqRNN(input_size=300)
        self.classification_head = nn.Linear(200, embedding.weight.shape[0])
    def forward(self, data):
        output = self.model(
              self.embedding(data['history']),
              self.embedding(data['answer']),
              data['history_len'],
              data['answer_len'])
        y = output["output"]
        return self.classification_head(y)


    def generate(self, data, max_size=100, init_token=0):
        with torch.no_grad():
            generated_tokens = []
            encoder_output_h, encoder_output_c =\
                self.model.encode(self.embedding(data['history']), data['history_len'])
            input_embed = self.embedding(torch.LongTensor([[init_token]]))
            for i in range(max_size):
                encoder_output_h, encoder_output_c = self.model.decode(input_embed, encoder_output_h, encoder_output_c)
                token = self.classification_head(encoder_output_h).argmax()
                generated_tokens.append(token.item())
                init_embed = self.embedding(token.unsqueeze(0))
        return generated_tokens

In [ ]:
from tqdm.notebook import trange, tqdm
from torch import optim
from torch import nn
from model import IntentRNN

class Trainer():
    def __init__(self, model, padding_idx=100):
        self.model = model
        self.padding_idx = padding_idx
        self.optimizer = None

    def before_training_loop(self, learning_rate = 1e-3):
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.criterion = nn.CrossEntropyLoss(ignore_index=5089)

    def validation_step(self, data):
        pass
    def training_step(self, data):
        y_pred = self.model(data)

        y_truth = data["answer"][:, 1:].flatten()

        loss_reconstruction = self.criterion(y_pred[:,:-1].reshape(y_truth.shape[0], -1), y_truth)
        (loss_reconstruction).backward()
        return loss_reconstruction.item()

    def on_validation_end(self, resp):
        pass

    def validation(self, validation_dl):
        pass



    def fit(self,
            training_dl,
            validation_dl,
            learning_rate = 1e-3,
            validation_frequency = 8,
            max_iter = 10000,
            gpu=False,

        ):
        if(gpu):
          self.model = self.model.cuda()
        self.before_training_loop(learning_rate)

        iter_count = 0
        loss_buffer = []
        pbar = trange(max_iter)

        while(True):
            for data in training_dl:
                if gpu:
                    data = {k:v.cuda() for k, v in data.items()}
                self.optimizer.zero_grad()
                loss_buffer += [self.training_step(data)]
                self.optimizer.step()

                if(iter_count  % validation_frequency == 0):
                    print("Loss at iteration %s is %s"%(iter_count, np.mean(loss_buffer)))
                    self.validation(validation_dl)
                    loss_buffer = []

                iter_count += 1
                if(iter_count > max_iter):
                    return
                pbar.update(1)